### Simulation Vélib accélérée avec numba et jit

- Pas de limite de places dans chaque station
- Vélos mis au hasard au départ

In [14]:
from numpy import *
import numpy as np
from numba import *

R=np.array([[0,0.22,0.32,0.2,0.26],[0.17,0.,0.34,0.21,0.28],[0.19,0.26,0.,0.24,0.31],[0.17,0.22,0.33,0.,0.28],[0.18,0.24,0.35,0.23,0.]])
T=np.array([[0,3.,5.,7.,7.],[2.,0,2.,5.,5.],[4.,2.,0,3.,3.],[8.,6.,4.,0,2.],[7.,7.,5.,2.,0]])
lambda_=np.array([2.8,3.7,5.5,3.5,4.6])/60.

@jit
def pickState(p):
    r=np.cumsum(p)
    r =r/np.sum(p)
    u=np.random.rand()
    w=0
    while(u>r[w]):
        w+=1
    return w

@jit
def transition(etat):
    Tpos=np.zeros(25)
    for i in np.arange(5):
        for j in np.arange(5):
            if (i==j):
                if (etat[5*i+j]>0):
                    Tpos[6*i]=lambda_[i]
            else:
                Tpos[5*i+j]=etat[5*i+j]*1./T[i,j]
    return Tpos

@jit
def saut(etat):
    t=transition(etat)
    tau=np.sum(t)
    t=t/tau
    x=pickState(t)
    #x=np.random.choice(len(t),p=t)
    depart=x//5
    arrivee=x%5
    if (depart==arrivee):
        #depart de la station depart
        # il faut choisir la station destination
        #z=np.random.choice(5,p=R[depart,:])
        z=pickState(R[depart,:])
        caseADiminuer = 6*depart
        caseAAugmenter =5*depart+z
        #print("Départ de "+str(depart)+" vers "+str(z)+"\n")
    else:
        # un vélo arrive 
        caseADiminuer=x
        caseAAugmenter= 6*arrivee
        #print("Arrivée en "+ str(arrivee)+ " en provenance de "+str(depart)+"\n")
    #print(etat.reshape(5,5))
    return -log(np.random.rand())/tau,caseADiminuer,caseAAugmenter

@jit
def trajectoire(etat,horizon):
    Timer=0.
    Tvide=np.zeros(5)
    while(Timer <horizon):
        t,x,y=saut(etat)
        Timer +=t
        for i in np.arange(5):
            if etat[6*i]==0:
                Tvide[i]+=t
        etat[x]-=1
        etat[y]+=1
    return Tvide/horizon

@jit
def simulation(Einitial,Nbiter,Horizon):
    #print("Durée ",Horizon)
    #print("Nombre d'itérations ",Nbiter)
    stationsVides=np.empty(5)
    for k in np.arange(Nbiter):
        stationsVides=np.vstack((stationsVides,trajectoire(Einitial,Horizon)))
    print("Pourcentage du temps où les stations sont vides\n")
    for i in np.arange(5):
        m=np.sum(stationsVides[:,i])/Nbiter
        print('Station '+str(i)+' : {:.3f}'.format(m))
        v=1.96*np.std(stationsVides[:,i])/np.sqrt(Nbiter)
        print('Intervalle de confiance : [{:.3f}'.format(m-v)+', {:.3f}'.format(m+v)+']')
        print("-------------\n")

Einitial=np.zeros(25)
Einitial[0]=1
print("Simulation avec 1 seul vélo\n")
simulation(Einitial,1000,10000.)

# Test de l'accord des valeurs avec Einitial=[1,0,0,...]
alpha=np.zeros((25,25))
for i in np.arange(5):
    for j in np.arange(5):
        alpha[6*i,5*i+j]=lambda_[i]*R[i,j]
        alpha[6*i,6*i]=-lambda_[i]
for i in np.arange(5):
    for j in np.arange(5):
        if (i!=j):
            alpha[5*i+j,5*i+j]=-1./T[i,j]
            alpha[5*i+j,6*j]=1./T[i,j]
        
alpha=np.transpose(alpha)
alpha=np.vstack((alpha[0:24,:],np.ones(25)))
beta=np.zeros(25)
beta[24]=1
x=np.linalg.solve(alpha,beta)[(np.arange(25)%5==np.arange(25)//5)]
print("Solution des équations de trafic, méthode algébrique\n")
print(1-x)

print("--------------------\n Simulation avec 100 vélos répartis au hasard")
import time
Einitial=np.zeros(25)
for i in np.arange(91):
    Einitial[np.random.randint(25)]+=1
a=time.time()
simulation(Einitial,1000,9000.)
#print(time.time()-a)



Simulation avec 1 seul vélo

Pourcentage du temps où les stations sont vides

Station 0 : 0.833
Intervalle de confiance : [0.831, 0.835]
-------------

Station 1 : 0.840
Intervalle de confiance : [0.838, 0.842]
-------------

Station 2 : 0.859
Intervalle de confiance : [0.858, 0.861]
-------------

Station 3 : 0.839
Intervalle de confiance : [0.837, 0.841]
-------------

Station 4 : 0.852
Intervalle de confiance : [0.850, 0.854]
-------------

Solution des équations de trafic, méthode algébrique

[0.83217043 0.83885467 0.85801393 0.83851615 0.85011589]
--------------------
 Simulation avec 100 vélos répartis au hasard
Pourcentage du temps où les stations sont vides

Station 0 : 0.010
Intervalle de confiance : [0.008, 0.012]
-------------

Station 1 : 0.046
Intervalle de confiance : [0.042, 0.049]
-------------

Station 2 : 0.160
Intervalle de confiance : [0.157, 0.163]
-------------

Station 3 : 0.044
Intervalle de confiance : [0.041, 0.048]
-------------

Station 4 : 0.115
Intervalle 

In [6]:
x

array([0.16782957, 0.16114533, 0.14198607, 0.16148385, 0.14988411])

In [7]:
1-x

array([0.83217043, 0.83885467, 0.85801393, 0.83851615, 0.85011589])

In [10]:
y=np.array([[0.166,0.160, 0.141 ,0.160 ,0.149]])
1-y

array([[0.834, 0.84 , 0.859, 0.84 , 0.851]])